# Исследование кредитоспособности заёмщиков в банке

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [ ]:
# загрузка библиотек
import pandas as pd
from pymystem3 import Mystem
m = Mystem()

In [37]:
# загрузка файла
df = pd.read_csv('/datasets/data.csv')

# получение общей информации о данных с помощью методов .info() и .head()
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


## Шаг 2. Предобработка данных

### Обработка пропусков

In [38]:
#Создание "словаря" с медианой 'total_income', груп. по категориям из 'income_type'
income_dict = dict(df.groupby('income_type')['total_income'].median())

#Заменяем пропущенные значения в столбце 'total_income' значениями из словаря 'income_dict'
df['total_income'] = df['total_income'].fillna(df['income_type'].apply(lambda x: income_dict.get(x)))
     #Check:
#print(df.head(15))
#df.info()

#Делаем то же самое для столбца days_employed, но без словаря, т.к. кол-во дней занятости не зависит от других параметров в таблице
df['days_employed'] = df['days_employed'].fillna(df['days_employed'].median())
     #Check:
#print(df.head(15))
#df.info()

**Вывод** : Пропуски в датасете заполнены медианой, так как мы не знаем точные данные и не можем проверить причину их отсутствия, а удалять их из таблицы - подвергать себя потере ценной информации

1. Принцип заполнения пропусков: в пропусках типа количественных переменных бывает большая разница в значениях. В таком случае, среднее арифметическое может некорректно характеризовать данные, поэтому наиболее эффективный способ - это заполнить пропуски медианой, сперва отсортировав числа по-порядку и разделив их по категориям из столбца income_type (тип занятости), т.к. ур зп может сильно отличаться у разных должностей.
2. Так как days_employed не явл. обязательным параметром для достижения целей поставленной задачи банком, и так как их не имеет смысла сгруппировывать с другими столбцами, я заполнила этот столбец средним статистическим значением по числам в данном столбце, чтобы выровнять таблицу. 

### Замена типа данных

In [39]:
#Заменяем вещественный тип данных на целочисленный
df[['days_employed', 'total_income']] = df[['days_employed', 'total_income']].astype('int')
#Check
#df.info()

**Вывод** : Замена вещественного типа данных на целочисленный не обязательна, но желательна в виду удобства и наглядности

1. Вещественный тип данных - тип float. Всего в данных 2 столбца, которые содержат entries данного типа:  'days_employed' (трудовой стаж в днях) и 'total_income' (доход в месяц). Заменяем значения на целочисленные для краткости и наглядности.
2. Для изменения типа данных был использован метод astype из-за его краткости и простоты использования

### Обработка дубликатов

In [40]:
#Сначала приведем столбцы с данными типа str в нижний регистр
df['education'] = df['education'].str.lower()
#Проверка на наличие разницы в регистрах в других столбцах (не найдено - продолжаем)
#print(df['insert_column'].unique())

#Подсчет кол-ва дубликатов в датафрейме(71)
#df.duplicated().sum()

#Удаление дубликатов + проверка
df = df.drop_duplicates().reset_index(drop= True)
#df.duplicated().sum()

**Вывод** : Найдены и устранены дубликаты в датасете

1. Для поиска и удаления дубликатов я использовала четыре метода: первый - найти уникальные значения в столбцах str, благодаря чему обнаружилось, что есть слова с одним значением, но разного регистра, которые считаются "уникальными"(это мешает поиску дубликатов), второй - исп. str.lower() метод для Pandas, чтобы привести строки в единый нижний регистр, третий - найти кол-во дубликатов двойным методом duplicated().sum(), четвертый - удалить дубликаты методом drop_duplicates.
2. Причины появления дубликатов. Их может быть много, например человеческий фактор (некорректное введение, ошибки при вводе данных, сокрытие информации, и т.д.) или технические ошибки, например неправильный свод данных их разных источников или баг

### Лемматизация

In [41]:
#Найдем уникальные лемматизированные значения в столбце 'purpose', чтобы использовать его в функции
lemmas = []
for words in df['purpose'].unique():
        lemmas_word = m.lemmatize(words)
        lemmas += lemmas_word
myset = list(set(lemmas))

#Так как список маленький, очистим его от пробелов, предлогов, прочего мусора вручную, оствив только описательные существительные
myset_clean = ['семья', 'образование', 'ремонт', 'покупка', 'строительство', 'автомобиль', 'жилье', 'операция', 'недвижимость', 'свадьба']

#Функция для лемматезации текста в столбце
def lemmatize_col(row):
    for word in m.lemmatize(row):
        if word in myset_clean:
            return word  
        
#Применение функции к столбцу в датафрейме
df['group_purpose'] = df['purpose'].apply(lemmatize_col)

#Проверка на наличие None
#df.isna().sum()

1. Процесс лемматизации подробно описан в закомментированных (#) строках в коде выше. Note, что в ручную такая обработка myset возможна только если список получается маленьким. Если же нет, я бы воспользовалась NLTK и сделала еще один loop по 'myset', где он выберет из списка слова которые == ‘NN’（noun).
2. словарь для лемматезации был собран из уникальных значение самого лемматизируемого столбца, чтобы увеличить точность при анализе данных

### Категоризация данных

In [43]:
# Проверка на кол-во респондентов по кол-ву детей в семье
#print(df['children'].value_counts())

#Замена значений 20 и -1 на релевантные значения в таблице (по логике: 20 - имелось в виду 2, (-1) - имелось в виду 0)
df['children'] = df['children'].mask(df['children']==20, 2) 
df['children'] = df['children'].mask(df['children']==(-1), 0) 

#print(df['family_status'].unique())

#Сборка всех данных в пивот таблицу и суммирование значение в 'TOTAL' для наглядности
data_pivot = df.pivot_table(index=['family_status'], columns='children', values='debt', aggfunc='sum',fill_value = 'N/A', margins = True, margins_name='TOTAL')
print(data_pivot)

children                    0      1      2     3    4    5  TOTAL
family_status                                                     
Не женат / не замужем   210.0   52.0   10.0   1.0  1.0  N/A    274
в разводе                55.0   21.0    8.0   1.0  0.0  N/A     85
вдовец / вдова           53.0    7.0    3.0   0.0  0.0  N/A     63
гражданский брак        229.0  118.0   33.0   8.0  0.0    0    388
женат / замужем         517.0  246.0  148.0  17.0  3.0    0    931
TOTAL                  1064.0  444.0  202.0  27.0  4.0    0   1741


**Вывод** : При просмотре данных о кол-ве детей у респондента выяснилось, что всего в таблице пришлось 76 значений на 20 детей и 47 значений на -1, а все остальные распределены между 0-5 детей. Из этого я сделала вывод, что при вводе данных произошла ошибка, и те, кто написали 20 - дописали лишний 0, а при -1 подразумевалось 0. Спросить о достоверности данного вывода нет возможности, поэтому вместо удаления данных строк, я воспользовалась данной логикой при замене. В итоге, я собрала интересующие нас числа в PivotTable.

1. Основная задача - разобраться, влияет ли семейное положение ('family_status') и количество детей ('children') клиента на факт возврата кредита в срок ('debt'). Процесс категоризации описан в самом задании в закомментированных (#) строках в коде.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [45]:
df_example = df.pivot_table(index = 'children', values = 'debt', 
                            aggfunc = ['count', 'sum', 'mean', lambda x: 1 - x.mean()])
df_example.columns = ['Кол-во пользователей', 'Кол-во должников', '% должников', '% НЕдолжников']
df_example.style.format({'% должников': '{:.2%}', '% НЕдолжников': '{:.2%}'})

,Кол-во пользователей,Кол-во должников,% должников,% НЕдолжников
children,,,,
0,14138,1064,7.53%,92.47%
1,4808,444,9.23%,90.77%
2,2128,202,9.49%,90.51%
3,330,27,8.18%,91.82%
4,41,4,9.76%,90.24%
5,9,0,0.00%,100.00%


**Вывод**

Не обязательно. Из таблицы понятно, что у большинства опрошенных (14 138 людей) детей нет, а должников среди них только 1064. Наибольший процент должников у категории людей с 4мя детьми(9.8% или 4 человека), но среди них был только 41 респондент. Интересно, что при этом у категории людей с 5 детьми (всего 9 человек) нет долгов. Из данных результатов я бы сделала вывод, что на % должников влияют не дети или не только дети, и важно рассмотреть также другие факторы. Тип займа в банке тоже имеет значение.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Да. Исходя из PivotTable в шаге 2.5, и результатам TOTAL по строкам (то есть, не важно, сколько детей у человека, смотрим только на семейное положение), можно сделать вывод, что кол-во женатых / замужних с долгами больше, значительно больше, чем у всех остальных, однако у не женатых / не замужних долгов больше, чем у тех, кто в разводе, вдова/вдовец или в гражданском браке. 

|family_status|TOTAL |  
|---|---|
|Не женат / не замужем |  **274**|
|в разводе     |         **85**|
|вдовец / вдова     |     **63**|
|гражданский брак |       **388**|
|женат / замужем    |     **931**|

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [55]:
#По официальным данным от Росстата, среднемесячная ЗП населения России в 2020 51083. 
#Для того чтобы считаться средним классом, в Москве необходимо зарабатывать от 121 тыс. руб

#Посчитаем, что средний ур. ЗП это приблизительно 51000-121000тыс руб. Ниже 51к - ниже ср., Выше 120к - выше ср.
def total_income_category(value):
    if value < 51000:
        return 0
    elif 51000 <= value <= 121000:
        return 1
    return 2 
df['income_level_index'] = df['total_income'].apply(total_income_category)

#Сделаем еще 1 PivoTable, разделив должников по гендерному признаку:
data_pivot = df.pivot_table(index=['income_level_index'], columns=['gender'], values='debt', aggfunc=['sum'],fill_value = 'N/A', margins = True, margins_name='TOTAL')
print(data_pivot)



                      sum                  
gender                  F      M  XNA TOTAL
income_level_index                         
0                    18.0    8.0  N/A    26
1                   386.0  186.0  N/A   572
2                   590.0  553.0    0  1143
TOTAL               994.0  747.0    0  1741


**Вывод**

Из Pivot Table выше можно сделать следующие выводы: 

1. Кол-во людей с ур. дохода выше среднего (индекс 2) гораздо больше тех, у кого ур. дохода ниже (1 и 0). Всего есть 6 должников с уровнем дохода ниже среднего (индекс 0)
                 
|income_level_index|TOTAL|                       
|---|---|
|0|**6**|
|1|**572**|
|2|**1143**|

2. Дополнительно, так как я отсортировала должников по гендерному признаку, можно сделать вывод, что кол-во должников женщин с доходом выше среднего, чем у мужчин с доходом выше среднего. То же самое относится к среднему и ниже среднего уровням дохода. 

|income_level_index|Female|Male|XNA  
|---|---|---|---|                         
|0|18.0|8.0|N/A  
|1|386.0|186.0|N/A   
|2|590.0|553.0|0   
|**TOTAL**|**994.0**|**747.0**|**0**   


- Как разные цели кредита влияют на его возврат в срок?

In [51]:
#Создадим еще 1 PivotTable
data_pivot = df.pivot_table(index=['group_purpose'], columns='gender', values='debt', aggfunc='sum',fill_value = 'N/A', margins = True, margins_name='TOTAL')
print(data_pivot)

gender             F      M  XNA  TOTAL
group_purpose                          
автомобиль     172.0  105.0  N/A    277
жилье           31.0   15.0  N/A     46
недвижимость    23.0   19.0  N/A     42
образование    207.0  163.0  N/A    370
операция       107.0   98.0  N/A    205
покупка        248.0  188.0    0    436
ремонт          23.0   12.0  N/A     35
свадьба        108.0   78.0  N/A    186
строительство   75.0   69.0  N/A    144
TOTAL          994.0  747.0    0   1741


**Вывод**

Из Pivot Table выше можно сделать следующие выводы: 
1. Больше всего должников среди двух имеющихся гендеров - те, кто берут кредит с целью покупки чего-либо (специфически женщин должников больше, чем мужчин). На 2 месте среди целей для кредита - образование, на 3 - автомобиль. Во всех случаях кол-во должников женщин больше, чем мужчин.

|group_purpose|TOTAL|
|---|---|
|автомобиль   |  277|
|жилье    |      46|
|недвижимость |  42|
|образование   | 370|
|операция    |   205|
|покупка   |     436|
|ремонт    |     35|
|свадьба   |     186|
|строительство|  144|


## Шаг 4. Общий вывод

Разберем итоги на несколько этапов:

1)После общей оценки состояния датасета, были выявлены несколько очевидных проблем: шибки в данных, разные регистры итд. 2. Возможные причины появления пропусков: чаще всего причиной является человеческий фактор. В данном случае, это может быть сокрытие личной информации (о занятости и уровне дохода). Другой возможной причиной могут быть ошибки ввода данных.

2)Пропуски в столбце 'total_income' были заполнены медианой после разделения по группам по доходу('income_type'), т.к. есть среди данных есть неравномерное распределение среди небольших и крупных значений, что может повлиять на точность анализа. Столбец 'days_employed' был заполнен средним статистическим значением в пропусках. Из-за ошибки формате/вводе данных в данном столбце, он не использовался при дальнейшем анализе.

3) Был заменен вещественный тип данных - тип float - на целочисленный. Всего в данных 2 столбца, которые содержат entries данного типа: 'days_employed' (трудовой стаж в днях) и 'total_income' (доход в месяц). Заменяем значения на целочисленные для краткости и наглядности.

4)Для поиска и удаления дубликатов было использовано четыре метода: первый - найти уникальные значения в столбцах str, благодаря чему обнаружилось, что есть слова с одним значением, но разного регистра, которые считаются "уникальными"(это мешает поиску дубликатов), второй - исп. str.lower() метод для Pandas, чтобы привести строки в единый нижний регистр, третий - найти кол-во дубликатов двойным методом duplicated().sum(), четвертый - удалить дубликаты методом drop_duplicates.

Причин появления дубликатов может быть много, например: человеческий фактор (некорректное введение, ошибки при вводе данных, сокрытие информации, и т.д.) или технические ошибки, например неправильный свод данных их разных источников или баг

5)При просмотре данных о кол-ве детей у каждого респондента выяснилось, что всего в таблице пришлось 76 значений на 20 детей и 47 значений на -1, а все остальные распределены между 0-5 детей. Из этого я сделала вывод, что при вводе данных произошла ошибка, и те, кто написали 20 - дописали лишний 0, а при -1 подразумевалось 0. Спросить о достоверности данного вывода нет возможности, поэтому вместо удаления данных строк, я воспользовалась данной логикой при замене. В итоге, я собрала интересующие нас числа в PivotTable

***ИТОГ***: 
- Семейное положение и количество детей клиента не влияют на выплату кредита, процент должников в каждой группе разделенной по количеству детей - около 9% (за исключении группы с 5ю детьми). При этом, среди женатых/замужних клиентов без детей должников в 2 раза больше, чем у любых других категорий. 

|family_status/children|0|1|2|3|4|5|  
|---|---|---|---|---|---|---|
|Не женат / не замужем|210.0|52.0|10.0 |1.0|1.0|N/A   |
|в разводе | 55.0 |21.0  |  8.0|   1.0 | 0.0|  N/A    |
|вдовец / вдова   |       53.0   | 7.0  |  3.0 |  0.0 | 0.0|  N/A|     
|гражданский брак   |     229.0 | 118.0  | 33.0|  8.0 | 0.0 |   0 |   
|женат / замужем   |      517.0  |246.0 | 148.0  |17.0|  3.0  |  0   |

- Также не обнаружена связь между целями кредита и своевременным погашением кредита.

**Рекоммендации**: 1) Уточнить формат данных в столбце 'days_employed' и перевести числа в корректную форму. 
2) необходим более повторный анализ данных с учетом множества других факторов, которые помогут ответить на вопрос влияет ли кол-во детей на наличие просрочек выплаты в банк. Конкретно, добавить в опрос несколько более подробных вопросов, прим.: 1) Есть ли в семье люди с инвалидностью/тяжелой болезнью? 2) Имеются ли долги в других банках/источниках? 3) Какие в среднем расходы на ребенка в месяц? итд